In [1]:
# from src.open_r1.trl_main.trl.trainer import SFTTrainer
from trl import SFTTrainer
from datasets import load_dataset, Dataset
import json
import pandas as pd
import pyarrow.parquet as pq
# from swanlab.integration.huggingface import SwanLabCallback
from peft import LoraConfig, TaskType, get_peft_model
import pyarrow.parquet as pq
import pandas as pd
import re
# 该代码多卡SFT 设置 cuda visiable 
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

/home/vipuser/miniconda3/envs/schema-R1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
# file_path = "training_prompt2.csv"
# model_path = "/home/LLM/models/Qwen2.5-0.5B-Instruct"
# model_path = "/home/LLM/models/Qwen2.5-0.5B-Instruct"
model_path = "/home/model/para_Qwen"

In [3]:
# Q
# Check if GPU benefits from bfloat16
if torch.cuda.get_device_capability()[0] >= 8:
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16


model_kwargs = dict(
    attn_implementation="sdpa",  # 注意力实现机制，可以使用flash_attention_2代替
    torch_dtype=torch_dtype, # 使用的torch dtype类型，默认为自动
    use_cache=False,  # 我们使用梯度检查点
    device_map="auto", # define model kwargs
)

model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=model_kwargs['torch_dtype'],
    bnb_4bit_quant_storage=model_kwargs['torch_dtype'],
)

In [4]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(model_path, **model_kwargs)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████| 4/4 [00:45<00:00, 11.49s/it]


In [9]:
# 拆解训练集
# 提供每次训练token数
from tqdm import tqdm
import copy 
import re
#  1600 8   2200 6   2600 
max_token = 1100
#  保证特殊字符能被识别
accept_toekn = max_token - 170
#获取REF表数据
tab_str = ""
schema_slicing = []
df = pd.read_csv("./dataset2/table_schema_Reference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['Reference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"

    
print(len(schema_slicing))

# print(ReF_group[5])
# print("#############################")
# print(ReF_group[6])


#获取noREF表数据
df = pd.read_csv("./dataset2/table_schema_noReference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['noReference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
#         print("+++++++++++")
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"
        
        
if len(tab_str) > 0:
    schema_slicing.append(tab_str)

print(len(schema_slicing))
# print(schema_slicing[6])
# print("#############################")
# print(schema_slicing[7])
# print(schema_slicing[8])

create_table_pattern = re.compile(r'CREATE TABLE `[^`]+` \([^;]+\);')
table_name_group = []
test_total = set()
count = 0
for piece in schema_slicing:
    create_table_statements = create_table_pattern.findall(piece)
    temp_list = []
    for table in create_table_statements:
        table_name = re.search(r"CREATE TABLE `([^`]+)`", table).group(1)
        temp_list.append(table_name.lower())
        test_total.add(table_name.lower())
        count+=1
    print(temp_list)
    table_name_group.append(temp_list)
# print(count)
print(len(test_total))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 1270.90it/s]


6


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 1082.57it/s]

8
['manufacturers', 'products', 'student', 'plays_games', 'sportsinfo', 'actor', 'entrepreneur', 'people', 'ref_hotel_star_ratings', 'ref_attraction_types', 'hotels', 'tourist_attractions', 'street_markets', 'shops', 'museums', 'royal_family', 'theme_parks', 'visits', 'photos', 'staff']
['tourist_attraction_features', 'wrestler', 'elimination', 'business', 'category', 'checkin', 'neighbourhood', 'review', 'tip', 'ref_detention_type', 'ref_incident_type', 'addresses', 'students', 'teachers', 'assessment_notes', 'behavior_incident', 'detention']
['student_addresses', 'students_in_detention', 'film', 'film_market_estimation', 'catalogs', 'catalog_structure', 'catalog_contents', 'catalog_contents_additional_attributes', 'routes', 'airports', 'stadium', 'game', 'injury_accident', 'physician', 'department', 'affiliated_with', 'trained_in', 'patient', 'nurse']
['appointment', 'prescribes', 'block', 'room', 'on_call', 'stay', 'undergoes', 'buildings', 'office_locations', 'region', 'party', 'me

In [10]:
def make_system_prompt(schema_pad):
    return f" I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema_pad}"

def make_user_prompt(question_pad, selected_table_pad):
    return f"{question_pad}######selected_table: {selected_table_pad}."

def make_reponse(answer_pad):
    return f"```Reference Table\n-- Tables: {answer_pad};\n```"

In [11]:
def make_conversation(schema_pad, question_pad, selected_table_pad,answer_pad):
    conversation = []
    conversation0 = {}
    conversation1 = {}
    conversation2 = {}
    system_prompt = make_system_prompt(schema_pad)
    user_prompt = make_user_prompt(question_pad, selected_table_pad)
    assistance_prompt = make_reponse(answer_pad)
    conversation0["role"] = "system"
    conversation0["content"] = system_prompt
    conversation1["role"] = "user"
    conversation1["content"] = user_prompt + "\n"
    conversation2["role"] = "assistant"
    conversation2["content"] = assistance_prompt
    conversation.append(conversation0)
    conversation.append(conversation1)
    conversation.append(conversation2)
    all_info = system_prompt + user_prompt + assistance_prompt
#     print(conversation2["content"])
    return conversation, all_info

In [12]:
def fill_dataset_template(schema_pad,question_pad,selected_table_pad,answer_pad):
     message = {
                "instruction": f" I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema_pad}",
                "input": f"{question_pad}######selected_table: {selected_table_pad}.",
                "output": f"```Reference Table\n-- Tables: {answer_pad};\n```",
            }
     return message

In [13]:
df = pd.read_csv("./dataset2/full_finetuning_dataset_cropped.csv",encoding="utf-8")
outer_index = 0
message_group = []
trans_data = {}
check1 = 0
for i in range(len(df['db_id'])):
    question = df['question'][i]
    query = df['query'][i]
    correct_table = df['correct_tables'][i]
    correct_table_list = correct_table.split(", ")
    dict_target_table_slice = {}
    for target_table in correct_table_list:
        for index, table_name_slicing in enumerate(table_name_group): 
            if target_table.lower() in table_name_slicing:
                dict_target_table_slice[target_table] = index
                break    
#     if  len(dict_target_table_slice) != len(correct_table_list):
#         print(f"error--{outer_index}----",list(dict_target_table_slice.keys()),"=====",correct_table_list)
#     outer_index += 1     
#     基于定位构建两条数据集: 一条顺序 一条判断
#     顺序
    sorted_relation = sorted(dict_target_table_slice.items(), key=lambda x: x[1])
    selected_table_list = []
    for index, piece in enumerate(schema_slicing):
        this_selected = []
        for key, value in sorted_relation:
            if index == value:
                this_selected.append(key)
#          给与选择提示
        message = None
        if len(selected_table_list) > 0:
            selected_table_str = ', '.join(selected_table_list)
            if len(this_selected) == 0:
                message = fill_dataset_template(piece,question,selected_table_str,"#None#")
                conversation, all_info = make_conversation(piece,question,selected_table_str,"#None#")
            else:
                this_selected_str = ', '.join(this_selected)
                message = fill_dataset_template(piece,question,selected_table_str,this_selected_str)
                conversation, all_info = make_conversation(piece,question,selected_table_str,this_selected_str)
        if message is not None:
            check1 += 1
            message_group.append(conversation)
            token_len = len(tokenizer(all_info, add_special_tokens=False)["input_ids"])
            if  token_len >  max_token:
                max_token = token_len
#          不给与选择提示 + 顺序决策   
        if len(this_selected) == 0:
            conversation, all_info = make_conversation(piece,question,"None","#None#")
            message = fill_dataset_template(piece,question,"None","#None#")
        else:
            this_selected_str = ', '.join(this_selected)
            message = fill_dataset_template(piece,question,"None",this_selected_str)
            conversation, all_info = make_conversation(piece,question,"None",this_selected_str)
        
        if len(this_selected) > 0:
            for item in this_selected:
                selected_table_list.append(item)
                
        message_group.append(conversation)
        check1 += 1
        
print(len(message_group))
trans_data["messages"] = message_group
print(check1)

20087
20087


In [14]:
print(max_token)

1100


In [15]:
trans_data = pd.DataFrame(trans_data)
trans_data = Dataset.from_pandas(trans_data,split="train")
print(len(trans_data))
for sample in trans_data:
    print(sample)
    break

20087
{'messages': [{'content': " I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:CREATE TABLE `Manufacturers` (\r\n  Code INTEGER,\r\n  Name VARCHAR(255),\r\n  Headquarter VARCHAR(255),\r\n  Founder VARCHAR(255),\r\n  Revenue REAL\r\n);\nCREATE TABLE `Products` (\r\n  Code INTEGER,\r\n  Name VARCHAR(255),\r\n  Price DECIMAL,\r\n  Manufacturer INTEGER REFERENCES Manufacturers(Code)\r\n);\nCREATE TABLE `Student` (\r\n  StuID INTEGER,\r\n  LName VARCHAR(12),\r\n  Fname VARCHAR(12),\r\n  Age INTEGER,\r\n  Sex VARCHAR(1),\r\n  Major INTEGER,\r\n  Advisor INTEGER,\r\n  city_code VARCHAR(3)\r\n);\nCREATE TABLE `Plays_Games` (\r\n  StuID INTEGER REFERENCES Student(StuID),\r\n  GameID INTEGER REFERENCES Video_Games(GameID),\r\n  Hours_Played INTEGER\r

In [16]:
# def fill_dataset_template(schema_pad,question_pad,selected_table_pad,answer_pad):
#      message = {
#                 "instruction": f" I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema_pad}",
#                 "input": f"{question_pad}######selected_table: {selected_table_pad}.",
#                 "output": f"```Reference Table\n-- Tables: {answer_pad};\n```",
#             }
#      return message
#  #None#

In [17]:
# df = pd.read_csv("./dataset2/full_finetuning_dataset_cropped.csv",encoding="utf-8")
# outer_index = 0
# message_group = []
# check2 = 0
# for i in range(len(df['db_id'])):
#     question = df['question'][i]
#     query = df['query'][i]
#     correct_table = df['correct_tables'][i]
#     correct_table_list = correct_table.split(", ")
#     dict_target_table_slice = {}
#     for target_table in correct_table_list:
#         for index, table_name_slicing in enumerate(table_name_group): 
#             if target_table.lower() in table_name_slicing:
#                 dict_target_table_slice[target_table] = index
#                 break    
# #     if  len(dict_target_table_slice) != len(correct_table_list):
# #         print(f"error--{outer_index}----",list(dict_target_table_slice.keys()),"=====",correct_table_list)
# #     outer_index += 1     
# #     基于定位构建两条数据集: 一条顺序 一条判断
# #     顺序
#     sorted_relation = sorted(dict_target_table_slice.items(), key=lambda x: x[1])
#     selected_table_list = []
#     for index, piece in enumerate(schema_slicing):
#         this_selected = []
#         for key, value in sorted_relation:
#             if index == value:
#                 this_selected.append(key)
# #          给与选择提示
#         message = None
#         if len(selected_table_list) > 0:
#             selected_table_str = ', '.join(selected_table_list)
#             if len(this_selected) == 0:
#                 message = fill_dataset_template(piece,question,selected_table_str,"#None#")
#             else:
#                 this_selected_str = ', '.join(this_selected)
#                 message = fill_dataset_template(piece,question,selected_table_str,this_selected_str)
#         if message is not None:
#             check2 += 1
#             message_group.append(message)
# #          不给与选择提示 + 顺序决策   
#         if len(this_selected) == 0:
#             message = fill_dataset_template(piece,question,"None","#None#")
#         else:
#             this_selected_str = ', '.join(this_selected)
#             message = fill_dataset_template(piece,question,"None",this_selected_str)
        
#         if len(this_selected) > 0:
#             for item in this_selected:
#                 selected_table_list.append(item)
                
#         message_group.append(message)
#         check2 += 1
        
# print(len(message_group))
# #     判断
# print(check2)

In [18]:
output_dir = "SFT_Q_S2"
num_train_epochs = 3
bf16 = True
overwrite_output_dir = True
per_device_train_batch_size = 2
gradient_accumulation_steps = 16
gradient_checkpointing = True
evaluation_strategy = "steps"
learning_rate = 5e-5
weight_decay = 0.01
lr_scheduler_type = "cosine"
warmup_ratio = 0.01
max_grad_norm = 0.3
group_by_length = True
auto_find_batch_size = False
save_steps = 50
logging_steps = 50
load_best_model_at_end= False
packing = False
save_total_limit=1
neftune_noise_alpha=5
# report_to="wandb"

In [19]:
lora_r = 64
lora_alpha = 32
lora_dropout = 0.1
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj'],
    task_type=TaskType.CAUSAL_LM,
)

In [20]:
from transformers import TrainingArguments
# from swanlab.integration.huggingface import SwanLabCallback
training_arguments = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    num_train_epochs=num_train_epochs,
    load_best_model_at_end=load_best_model_at_end,
    per_device_train_batch_size=per_device_train_batch_size,
#     evaluation_strategy=evaluation_strategy,
    max_grad_norm = max_grad_norm,
    auto_find_batch_size = auto_find_batch_size,
    save_total_limit = save_total_limit,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=bf16,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="none",
    neftune_noise_alpha= neftune_noise_alpha
)

In [21]:
# swanlab_callback = SwanLabCallback(
#     project="Qwen2.5_1.5B_GRPO_steer_CoT",
#     experiment_name="Qwen2.5_1.5B_GRPO_steer_CoT",
#     description="Qwen2.5_1.5B_GRPO_steer_CoT",
#     config={
#         "model": "Qwen2.5-1.5B",
#         "dataset": "steer_CoT",
#     },
# )

In [22]:
trainer = SFTTrainer(
    model= model,
    train_dataset=trans_data,
    peft_config = peft_config,
    args=training_arguments,
#     callbacks=[swanlab_callback],
#     params_mask = params_mask
)

# trainer.train()

Truncating train dataset: 100%|█████████████████████████████████████████████████████████████| 20087/20087 [00:00<00:00, 38712.98 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss


In [20]:
trainer.model.save_pretrained(output_dir)

In [21]:
import os
import shutil
from pathlib import Path
 
# 源文件路径
tokenizer_config_path = os.path.join(model_path, "tokenizer_config.json")
tokenizer_path = os.path.join(model_path, "tokenizer.json")
  
# 确保目标目录存在
Path(output_dir).mkdir(parents=True, exist_ok=True)
 
# 复制文件
try:
    shutil.copy2(tokenizer_config_path, output_dir)
    shutil.copy2(tokenizer_path, output_dir)
    print("文件复制成功！")
except FileNotFoundError as e:
    print(f"错误：找不到源文件 - {e}")
except Exception as e:
    print(f"发生错误：{e}")# inference 

文件复制成功！


In [ ]:
#  use_my_SFTtrainer 自己的trl库源码
# processing_class, train_dataset = trainer.get_dataset()
# print(train_dataset)
# print(processing_class)
# # completions_text = self.processing_class.batch_decode(completion_ids, skip_special_tokens=True)

In [16]:
# print(type(train_dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [17]:
# for i in train_dataset:
#     print(i)
#     break

{'source': 'aplaca', 'num_turns': 2, 'text': "<|im_start|>system\nYou are a helpful AI Assistant that provides well-reasoned and detailed responses. You first think about the reasoning process as an internal monologue and then provide the user with the answer. Respond in the following format: <think>\n...\n</think>\n<answer>\n...\n</answer><|im_end|>\n<|im_start|>user\nDescribe a process of making crepes.\n<|im_end|>\n<|im_start|>assistant\nMaking crepes is an easy and delicious process! Here are step-by-step instructions on how to make them:\n\n1. Assemble your ingredients. For basic crepes, you'll need: 1 cup all-purpose flour, 2 eggs, 1/2 cup milk, 1/2 cup water, 1/4 teaspoon salt, and 2 tablespoons melted butter.\n\n2. Mix the batter: In a large mixing bowl, whisk together the flour and the eggs. Gradually add the milk and water, stirring constantly to ensure that there are no lumps. Add salt and melted butter, and mix well.\n\n3. Let the batter rest: If you can, let the batter sit

In [14]:
# trainer.train()

swanlab: swanlab version 0.5.5 is available!  Upgrade: `pip install -U swanlab`    
swanlab: Tracking run with swanlab version 0.5.4                                   
swanlab: Run data will be saved locally in /home/LLM/code/open-r1-main/swanlog/run-20250409_133541-a3b1799d
swanlab: 👋 Hi winhong, welcome to swanlab!
swanlab: Syncing run Qwen2.5-1.5B_Full_Aplaca_test_GRPO_SFT to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@winhong/Qwen2.5-1.5B_Full_Aplaca_test_GRPO_SFT
swanlab: 🚀 View run at https://swanlab.cn/@winhong/Qwen2.5-1.5B_Full_Aplaca_test_GRPO_SFT/runs/wjtnv2yua7qkfkwpa4cmg


KeyboardInterrupt: 

In [20]:
output_dir = os.path.join("./", "/home/LLM/models/SFT_model/Qwen_1.5B_GRPO_test")
trainer.model.save_pretrained(output_dir)